In [4]:
import nibabel as nib
import os
import glob
import json
from pathlib import Path
import pandas as pd
import subprocess as sub 
from subprocess import Popen, PIPE, STDOUT

In [3]:
import numpy as np

In [23]:
label_df_lgg = pd.read_csv('/working/lupolab/julia/tcia_analysis/labels_tcia_lgg.csv')
label_df_gbm = pd.read_csv('/working/lupolab/julia/tcia_analysis/labels_tcia_GBM.csv')
full_label_df = label_df_lgg.append(label_df_gbm, ignore_index = True)

In [24]:
full_label_df['patient_id'] = [x.split('/')[7] for x in full_label_df.image]

In [25]:
patient_list = full_label_df.patient_id.unique()

In [26]:
usable_patients = []
for patient in patient_list: 
    patient_df = full_label_df.loc[(full_label_df['patient_id'] == patient) & (full_label_df['ground_truth'] != "OTHER")]
#     print(patient_df)
    if len(patient_df.ground_truth.unique()) >= 4: 
        usable_patients.append(patient)

In [27]:
usable_patient_df = full_label_df.loc[(full_label_df['patient_id'].isin(usable_patients)) & 
                                      (full_label_df['ground_truth'] != "OTHER")].copy()

In [28]:
root_dir = '/working/lupolab/julia/tcia_analysis'


In [33]:
usable_patient_df['aligned_image'] = ['' for x in usable_patient_df.image]

In [34]:
for idx, row in usable_patient_df.iterrows():
    dir_name = '/'.join(row['image'].split('/')[0:-1])
    image_name =row['image'].split('/')[-1]
    image_name_split = image_name.split('.')
    image_name_split[-3] = image_name_split[-3]+'a'
    image_name_aligned = '.'.join(image_name_split)
    if image_name_aligned in os.listdir(dir_name): 
        row['aligned_image'] = dir_name+'/'+image_name_aligned
    
    

In [35]:
def find_patient_images(patient_id, usable_patient_df): 
    images_dict = {}
    patient_images_df = usable_patient_df.loc[usable_patient_df.patient_id == patient_id]
    for x in ['T1C', 'T1', "T2", 'T2_FLAIR']: 
        contrast_images_df = patient_images_df.loc[patient_images_df.ground_truth == x].reset_index(drop = True)
        if contrast_images_df.shape[0] > 1: 
            for idx, row in contrast_images_df.iterrows(): 
                if 'ax'  in row['sd'].lower(): 
                    if row['aligned_image']: 
                        images_dict[x] =  '/'.join(row['aligned_image'].split('/')[6:])
                    else: 
                        images_dict[x] =  '/'.join(row['image'].split('/')[6:])
                    continue
                elif x not in images_dict.keys(): 
                    if row['aligned_image']: 
                        images_dict[x] =  '/'.join(row['aligned_image'].split('/')[6:])
                    else: 
                        images_dict[x] =  '/'.join(row['image'].split('/')[6:])
        else: 
            if contrast_images_df['aligned_image'][0]: 
                images_dict[x] = '/'.join(contrast_images_df['aligned_image'][0].split('/')[6:])
            else: 
                images_dict[x] = '/'.join(contrast_images_df['image'][0].split('/')[6:])
            
    return images_dict

In [36]:
x = list(usable_patient_df.patient_id.unique())

In [51]:
i =1 
for patient_id in x: 
    
    images_dict = find_patient_images(patient_id, usable_patient_df)
    
    values = list(images_dict.values())
    root = '/working/lupolab/julia/tcia_analysis/datasets/'
    image1 = nib.load(root+values[0])
    image2 = nib.load(root+values[1])
    image3 = nib.load(root+values[2])
    image4 = nib.load(root+values[3])
    
    check_sizes = image1.shape == image2.shape == image3.shape == image4.shape
    if not check_sizes: 
        print(i)
        print(patient_id)
        print(image1.shape, image2.shape, image3.shape, image4.shape)
    i += 1

44
TCGA-CS-4944
(219, 219, 150) (219, 219, 150) (220, 220, 300) (219, 219, 150)
46
TCGA-DU-A6S6
(240, 240, 175) (240, 240, 175) (240, 240, 70) (240, 240, 175)
104
TCGA-06-0156
(240, 240, 66) (240, 240, 66) (240, 240, 110) (240, 240, 165)
105
TCGA-76-6282
(219, 219, 150) (219, 219, 150) (220, 220, 300) (219, 219, 150)
168
TCGA-06-0165
(240, 240, 174) (199, 199, 150) (240, 240, 348) (240, 240, 174)
173
TCGA-76-4926
(219, 219, 150) (219, 219, 150) (219, 219, 300) (219, 219, 150)
199
TCGA-76-4932
(219, 219, 150) (219, 219, 150) (220, 220, 300) (219, 219, 150)
206
TCGA-06-0166
(240, 240, 168) (240, 240, 180) (240, 240, 114) (240, 240, 171)


In [14]:
fla = nib.load('/working/lupolab/julia/tcia_analysis/datasets/TCGA-LGG-nifti/TCGA-DU-7301/11-12-1991-MRI_BRAIN_WWO_CONTRAST-22050/10-AX_T1_POST_GD_FLAIR-56964a.nii.gz')
t1 = nib.load('/working/lupolab/julia/tcia_analysis/datasets/TCGA-LGG-nifti/TCGA-DU-7301/11-12-1991-MRI_BRAIN_WWO_CONTRAST-22050/8-AX_T1_pre_gd-40072a.nii.gz')
t1c = nib.load('/working/lupolab/julia/tcia_analysis/datasets/TCGA-LGG-nifti/TCGA-DU-7301/11-12-1991-MRI_BRAIN_WWO_CONTRAST-22050/5-AX_T2_FR-FSE_RF2_150-68026a.nii.gz')
t2 = nib.load('/working/lupolab/julia/tcia_analysis/datasets/TCGA-LGG-nifti/TCGA-DU-7301/11-12-1991-MRI_BRAIN_WWO_CONTRAST-22050/6-AXIAL_FLAIR-66613a.nii.gz')

In [15]:
fla.shape

(240, 240, 175)

In [16]:
t1.shape

(240, 240, 175)

In [17]:
t2.shape

(240, 240, 175)

In [18]:
t1c.shape

(240, 240, 175)

In [22]:
fla.shape == t1.shape == 

True